# 🧩 Cellule 1 – Environnement et imports

In [1]:
# ===============================================================
# 🧩 08_PixCell_LoRA_Conditionnel.ipynb
# Cellule 1 : Imports & configuration de l'environnement
# ===============================================================
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
from diffusers import DiffusionPipeline

# ---------------------------------------------------------------
# 🔧 Configuration générale
# ---------------------------------------------------------------
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"✅ Device actif : {device}")

# Répertoires du projet
PROJECT_ROOT = Path("/workspace")  # adapte selon ton environnement
DATA_DIR     = PROJECT_ROOT / "data" / "NCT-CRC-HE-100K"
CONFIG_DIR   = PROJECT_ROOT / "configs"
OUTPUT_DIR   = PROJECT_ROOT / "outputs" / "pixcell_lora"
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

print(f"📂 Dossier de sortie : {OUTPUT_DIR}")

# ---------------------------------------------------------------
# Vérification CUDA
# ---------------------------------------------------------------
if device.type == "cuda":
    print(f"GPU détecté : {torch.cuda.get_device_name(0)}")
    print(f"CUDA version : {torch.version.cuda}")
    print(f"PyTorch version : {torch.__version__}")
else:
    print("⚠️ Pas de GPU détecté — attention aux performances.")


✅ Device actif : cuda
📂 Dossier de sortie : /workspace/outputs/pixcell_lora
GPU détecté : NVIDIA GeForce RTX 4060 Laptop GPU
CUDA version : 12.4
PyTorch version : 2.4.0


# 🧩 Cellule 2 – Chargement de PixCell + UNI-2h

In [2]:
# ===============================================================
# 🧩 Cellule 2 : Chargement PixCell (avec VAE explicite) + UNI-2h (timm)
# ===============================================================
import os, shutil, torch, timm
import torch.nn.functional as F
from pathlib import Path
from huggingface_hub import login
from diffusers import DiffusionPipeline, AutoencoderKL
from timm.data import resolve_data_config
from timm.data.transforms_factory import create_transform

# ---------------------------------------------------------------
# 🔐 Authentification Hugging Face
# ---------------------------------------------------------------
HF_TOKEN = os.getenv("HF_TOKEN")
assert HF_TOKEN is not None, "⚠️ Variable d’environnement HF_TOKEN absente."
login(token=HF_TOKEN, add_to_git_credential=True)
print("🔑 Authentification Hugging Face réussie.")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
CACHE_DIR = Path(os.getenv("HF_HOME", "/workspace/.cache/huggingface"))

# ---------------------------------------------------------------
# 1️⃣ Chargement du pipeline PixCell-256 (avec VAE explicite)
# ---------------------------------------------------------------
def load_pixcell_with_explicit_vae(device):
    print("⏳ Chargement du VAE (SD 3.5 Large)…")
    vae = AutoencoderKL.from_pretrained(
        "stabilityai/stable-diffusion-3.5-large",
        subfolder="vae",
        torch_dtype=torch.float16,
        token=HF_TOKEN,
    ).to(device)
    print("✅ VAE chargé.")

    print("⏳ Chargement de PixCell-256…")
    pipe = DiffusionPipeline.from_pretrained(
        "StonyBrook-CVLab/PixCell-256",
        custom_pipeline="StonyBrook-CVLab/PixCell-pipeline",
        trust_remote_code=True,
        torch_dtype=torch.float16,
        vae=vae,
        token=HF_TOKEN,
    ).to(device)
    print("✅ PixCell-256 chargé.")
    return pipe

try:
    pipe256 = load_pixcell_with_explicit_vae(device)
except OSError as e:
    print("⚠️ Cache local corrompu :", e)
    model_cache_root = CACHE_DIR / "hub" / "models--StonyBrook-CVLab--PixCell-256" / "snapshots"
    if model_cache_root.exists():
        for snap in model_cache_root.iterdir():
            vae_dir = snap / "vae"
            if vae_dir.exists():
                print(f"🧹 Suppression du sous-dossier VAE corrompu : {vae_dir}")
                shutil.rmtree(vae_dir, ignore_errors=True)
    pipe256 = load_pixcell_with_explicit_vae(device)

# ---------------------------------------------------------------
# 2️⃣ Chargement du modèle UNI-2h (timm)
# ---------------------------------------------------------------
print("\n⏳ Chargement du modèle UNI-2h (via timm)…")

timm_kwargs = {
    "img_size": 224,
    "patch_size": 14,
    "depth": 24,
    "num_heads": 24,
    "embed_dim": 1536,
    "mlp_ratio": 2.66667 * 2,   # spécifique à UNI-2h
    "init_values": 1e-5,
    "num_classes": 0,
    "no_embed_class": True,
    "reg_tokens": 8,
    "dynamic_img_size": True,
    "mlp_layer": timm.layers.SwiGLUPacked,
    "act_layer": torch.nn.SiLU,
}

uni = timm.create_model(
    "hf-hub:MahmoodLab/UNI2-h",
    pretrained=True,
    **timm_kwargs
).eval().to(device)

cfg = resolve_data_config(uni.pretrained_cfg, model=uni)
tfm = create_transform(**cfg)

print("✅ UNI-2h prêt →", next(uni.parameters()).dtype, "| device:", device)

# ---------------------------------------------------------------
# 3️⃣ Fonction utilitaire : embeddings UNI-2h
# ---------------------------------------------------------------
@torch.inference_mode()
def uni_embed_from_tensor(x_3chw: torch.Tensor) -> torch.Tensor:
    """
    Extrait les embeddings UNI-2h d'une image (B,3,H,W)
    → Retourne (B,1536) en float32
    """
    if x_3chw.shape[-1] != 224:
        x_3chw = F.interpolate(x_3chw, size=(224, 224),
                               mode="bilinear", align_corners=False)
    e = uni(x_3chw).float()
    if hasattr(e, "shape") and e.ndim == 3:
        e = e[:, 0]  # token [CLS]
    return e

# ---------------------------------------------------------------
# 4️⃣ Vérification dtype/device PixCell
# ---------------------------------------------------------------
ref_weight = pipe256.transformer.pos_embed.proj.weight
PIPE_DTYPE = ref_weight.dtype
PIPE_DEV   = ref_weight.device
print(f"\nPixCell dtype : {PIPE_DTYPE}, device : {PIPE_DEV}")


Token has not been saved to git credential helper.
Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub.
Run the following command in your terminal in case you want to set the 'store' credential helper as default.

git config --global credential.helper store

Read https://git-scm.com/book/en/v2/Git-Tools-Credential-Storage for more details.
🔑 Authentification Hugging Face réussie.
⏳ Chargement du VAE (SD 3.5 Large)…
✅ VAE chargé.
⏳ Chargement de PixCell-256…


Keyword arguments {'trust_remote_code': True} are not expected by PixCellPipeline and will be ignored.


Loading pipeline components...:   0%|          | 0/3 [00:00<?, ?it/s]

The config attributes {'flow_shift': 1.0, 'use_flow_sigmas': False} were passed to DPMSolverMultistepScheduler, but are not expected and will be ignored. Please verify your scheduler_config.json configuration file.
The config attributes {'double_self_attention': False, 'num_vector_embeds': None, 'only_cross_attention': False, 'use_linear_projection': False} were passed to PixCellTransformer2DModel, but are not expected and will be ignored. Please verify your config.json configuration file.


✅ PixCell-256 chargé.

⏳ Chargement du modèle UNI-2h (via timm)…
✅ UNI-2h prêt → torch.float32 | device: cuda

PixCell dtype : torch.float16, device : cuda:0


# 🧩 Cellule 3 – Dataset, mappings et prototypes UNI-2h

In [5]:
# ===============================================================
# ⚙️ Configuration du chemin projet pour importer p9dg
# ===============================================================
import sys
from pathlib import Path

PROJECT_ROOT = Path("/workspace")  # 🔧 adapte si besoin (par ex. ".." en local)
PACKAGE_DIR = PROJECT_ROOT / "p9dg"

if str(PROJECT_ROOT) not in sys.path:
    sys.path.append(str(PROJECT_ROOT))

assert PACKAGE_DIR.exists(), f"❌ Dossier {PACKAGE_DIR} introuvable."
print(f"✅ Chemin ajouté au PYTHONPATH : {PROJECT_ROOT}")



✅ Chemin ajouté au PYTHONPATH : /workspace


In [9]:
# ✅ Détection robuste du split + init dataset sans doublon de chemin
from pathlib import Path
from p9dg.histo_dataset import HistoDataset

DATA_ROOT = Path("/workspace/data")            # <-- parent, pas le sous-dossier
assert DATA_ROOT.exists(), f"{DATA_ROOT} introuvable"

if (DATA_ROOT / "NCT-CRC-HE-100K").is_dir():
    SPLIT = "NCT-CRC-HE-100K"
elif (DATA_ROOT / "CRC-VAL-HE-7K").is_dir():
    SPLIT = "CRC-VAL-HE-7K"
else:
    raise FileNotFoundError("Ni 'NCT-CRC-HE-100K' ni 'CRC-VAL-HE-7K' trouvés sous /workspace/data")

print(f"🔎 Split détecté : {SPLIT}")
print(f"📁 Base effective : {DATA_ROOT / SPLIT}")

train_ds = HistoDataset(
    root_data=DATA_ROOT,
    split=SPLIT,
    apply_quality_filter=True,
    thresholds_json_path=SEUILS_PATH,   # chemin JSON (pas un dict)
    balance_per_class=True,
    samples_per_class_per_epoch=30,
)

print(f"✅ Dataset chargé ({len(train_ds)} images) | {len(train_ds.classes)} classes")
print("Classes :", train_ds.classes)


🔎 Split détecté : NCT-CRC-HE-100K
📁 Base effective : /workspace/data/NCT-CRC-HE-100K
🎨 Référence Vahadane fixée : TUM-TCGA-WAEEFPKC.tif
🎨 Référence Vahadane auto: TUM-TCGA-WAEEFPKC.tif
✅ Seuils par classe chargés depuis : /workspace/configs/seuils_par_classe.json
⚖️ Échantillonnage équilibré activé (30 images / classe).


AttributeError: 'HistoDataset' object has no attribute 'classes'

In [ ]:
# ===============================================================
# 🧩 Cellule 3 : Chargement du dataset + mappings + prototypes UNI-2h
# ===============================================================
from pathlib import Path
import json, torch
from p9dg.histo_dataset import HistoDataset
from utils.class_mappings import class_labels, make_idx_mappings

# ---------------------------------------------------------------
# 1️⃣ Chargement des seuils par classe
# ---------------------------------------------------------------
SEUILS_PATH = CONFIG_DIR / "seuils_par_classe.json"
with open(SEUILS_PATH, "r") as f:
    seuils_par_classe = json.load(f)
print(f"✅ Seuils par classe chargés depuis : {SEUILS_PATH}")

# ---------------------------------------------------------------
# 2️⃣ Initialisation du dataset équilibré
# ---------------------------------------------------------------
DATA_PATH = Path("/workspace/data/NCT-CRC-HE-100K")  # 🔧 chemin correct

train_ds = HistoDataset(
    root_data=DATA_PATH,
    apply_quality_filter=True,
    thresholds_json_path=SEUILS_PATH,        # chemin JSON (pas dict)
    balance_per_class=True,
    samples_per_class_per_epoch=30,          # pour un test rapide
)

print(f"✅ Dataset chargé ({len(train_ds)} images) | {len(train_ds.classes)} classes")
print("Classes :", train_ds.classes)

# ---------------------------------------------------------------
# 3️⃣ Mappings lisibles
# ---------------------------------------------------------------
idx_to_name, idx_to_color, class_to_label = make_idx_mappings(train_ds.class_to_idx)

# ---------------------------------------------------------------
# 4️⃣ Calcul ou rechargement des prototypes UNI-2h
# ---------------------------------------------------------------
PROTOS_PATH = OUTPUT_DIR / "prototypes_uni2h.pt"
if PROTOS_PATH.exists():
    PROTOS = torch.load(PROTOS_PATH)
    print(f"✅ Prototypes UNI-2h rechargés depuis {PROTOS_PATH}")
else:
    print("⏳ Calcul des prototypes UNI-2h par classe...")
    PROTOS = {}
    for cname in train_ds.class_to_idx.keys():
        imgs = [train_ds._resize(train_ds._load_image(p))
                for p in train_ds.paths_by_class[train_ds.class_to_idx[cname]][:8]]
        batch = torch.stack([train_ds._to_tensor(img).to(device) for img in imgs])
        e = uni_embed_from_tensor(batch)  # [B,1536]
        PROTOS[cname] = e.mean(0).detach().cpu()
    torch.save(PROTOS, PROTOS_PATH)
    print(f"✅ Prototypes UNI-2h sauvegardés → {PROTOS_PATH}")

# ---------------------------------------------------------------
# 5️⃣ Vérification rapide
# ---------------------------------------------------------------
for cname, v in PROTOS.items():
    print(f"{cname:<8} → {tuple(v.shape)}")
